## 作业描述
根据课上所学内容，在weibo_senti_100k情感分析数据集上使用Gensim包的CBOW算法训练词向量，并且使用Annoy建立词向量索引，加速搜索速度。


+ 认真阅读注释，对你做对该题至关重要

+ 作业已经给出大部分程序实现代码，你只需要在`######## your code ~n line ########` 与 `######## your code end ########` 行之间根据提示补充完毕相应的代码即可。

数据集信息

+ 下载地址：[百度网盘](https://pan.baidu.com/s/1DoQbki3YwqkuwQUOj64R_g#list/path=%2F)
+ 数据概览： 10 万多条，带情感标注 新浪微博，正负向评论约各 5 万条
+ 推荐实验： 情感/观点/评论 倾向性分析
+ 数据来源： [github](https://github.com/SophonPlus/ChineseNlpCorpus/blob/master/datasets/weibo_senti_100k/intro.ipynb)
+ 原数据集： 新浪微博，情感分析标记语料共12万条，网上搜集，具体作者、来源不详

### 1.数据集下载并解压
把下载后的文件放在与当前jupyter同级目录下的data文件中

### 2.读取文本内容

In [102]:
# 为了使下面的代码演示更加舒适，这里开启全局禁用警告提示
import warnings
warnings.filterwarnings("ignore")

+ 可以看到数据集有两列，第一列是标签列（对于训练词向量任务，没有用），1代表积极的情绪，0代表消极的情绪；第二列是review（评论）列，我们只使用标签列训练词向量。

In [103]:
import pandas as pd
path = './data/'
pd_all = pd.read_csv(path + 'weibo_senti_100k.csv')
pd_all.head()

,label,review
0,1,﻿更博了，爆照了，帅的呀，就是越来越爱你！生快傻缺[爱你][爱你][爱你]
1,1,@张晓鹏jonathan 土耳其的事要认真对待[哈哈]，否则直接开除。@丁丁看世界 很是细心...
2,1,姑娘都羡慕你呢…还有招财猫高兴……//@爱在蔓延-JC:[哈哈]小学徒一枚，等着明天见您呢/...
3,1,美~~~~~[爱你]
4,1,梦想有多大，舞台就有多大![鼓掌]


In [104]:
print('评论数目（总体）：%d' % pd_all.shape[0])
print('评论数目（正向）：%d' % pd_all[pd_all.label==1].shape[0])
print('评论数目（负向）：%d' % pd_all[pd_all.label==0].shape[0])

评论数目（总体）：119988
评论数目（正向）：59993
评论数目（负向）：59995


### 问题一：从pd_all中提取review列的数据，并且把每一行的数据存储到data中

In [105]:
######## your code ~n line ######## 提示：这里data是一个列表，列表中每一个元素就是一条评论，方法有多种，可自行发挥


data = 
######## your code end ########
'一共%s条数据.' % (len(data), )

'一共119988条数据.'

In [106]:
data[0]

'\ufeff更博了，爆照了，帅的呀，就是越来越爱你！生快傻缺[爱你][爱你][爱你]'

### 3.中文分词
这里我们使用jieba进行中文分词。

In [107]:
import jieba
# 实现分词函数
def tokenizer(text):
    #精确模式
    tokens = jieba.lcut(text, cut_all=False)
    return tokens

### 问题二：把数据处理成训练词向量需要的格式。

In [108]:
from tqdm import tqdm
sentence_tokens = []
for paragraph in tqdm(data):
    # 提示：对data中的每一个条评论进行分词，每个分词后的句子，词与词之间用空格隔开，并追加到sentence_tokens中
    ######## your code ~n line ########
    tokens = 
    ######## your code end line ########
    sentence_tokens.append(tokens)

100%|█████████████████████████████████| 119988/119988 [00:31<00:00, 3811.48it/s]


In [109]:
# sentence_tokens[10004]
'一共%s个句子.' % (len(sentence_tokens), )

'一共119988个句子.'

In [110]:
# 由于数据量比较大这里将其保存在文件中，每行一个句子，句子中词与词用空格分开
with open('sentence_tokens_zh.txt', 'w', encoding='utf-8') as f:
    f.write('\n'.join(sentence_tokens))

### 4.训练词向量

Gensim被称为“为人类进行主题建模”的自然语言处理包。但实际上远不止这些。它是一个领先和最先进的软件包，可以用于处理文本、文本向量（如Word2Vec、FastText等）和构建主题模型。

In [111]:
# 安装gensim
!pip install gensim==4.2.0

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [112]:
# Word2Vec 中有较多的参数需要指定，这里逐一进行说明，想知道更加细致的讲解，还是建议去看官方文档
param = {
    'sentences':None, # 是一个可以迭代的对象，对于大语料集，建议使用BrownCorpus,Text8Corpus或LineSentence构建。
    'corpus_file':None, # 语料的路径，一般与LineSentence配合使用来提升训练的性能
    'vector_size':100, # 词向量大小，默认100
    'alpha':0.025, # 初始学习速率
    'window':7, # 窗口大小，默认5
    'min_count':5, # 词频低于min_count以下的词舍去
    'max_vocab_size':None, # 最大的词典大小
    'sample':1e-3, # 配置哪些高频字随机降采样的阈值，取值范围(0, 1e-5)
    'seed':1, # 随机种子，可保证每次训练的结果一致
    'workers':6, # 训练使用的线程数量
    'min_alpha':0.0001, # 随着训练的进行，学习率将从“alpha”线性下降到“min_alpha”
    'sg':0, # 选择1使用skip-gram模型，0使用cbow模型
    'hs':0, # 选择1使用hierarchical softmax进行优化，选择0而且“negative”参数是非0，则使用负采样
    'negative':5, # 负采样词的个数
    'ns_exponent':0.75, # 用于形成负采样分布的指数
    'cbow_mean':1, # 如果为0，则使用上下文词向量之和。如果1，则使用平均值，仅在使用cbow时适用
    'hashfxn':hash, # 用于随机初始化权重的哈希函数，以提高训练再现性
    'epochs':5, # 训练的轮次
    'sorted_vocab':1, # 如果为1，则在指定单词索引之前，按频率降序对词汇进行排序。
    'batch_words':10000, # 每一批的传递给线程的单词的数量，默认为10000
    'compute_loss':False, # 如果是True计算并存储损失值；可以使用gensim.models.word2vec.Word2Vec.get_latest_training_loss方法获取
    'max_final_vocab':None, # 限制目标词表的大小
    'shrink_windows':True,  # 有效窗口大小从[1，window]均匀采样
}

In [113]:
# 从gensim中导出Word2Vec包
from gensim.models.word2vec import Word2Vec, LineSentence

### 问题三：使用LineSentence实现大规模文本的读取

In [114]:
# 1.训练模型
filename = 'sentence_tokens_zh.txt'
# 提示：初始化LineSentence类，赋值给text。[LineSentence类官方教程](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.LineSentence)
####### your code ~1 line ######## 
text = 
####### your code end ######## 
param['sentences'] = text
wv_model = Word2Vec(**param)

In [115]:
# 2.保存模型
# binary设置为True，训练的词向量将会以二进制的方式存储
wv_model.wv.save_word2vec_format('./wv_model/word2vec_zh.txt', binary=False)

### 问题四：加载训练的词向量。

In [116]:
# 3.加载模型（词向量）
from gensim.models import KeyedVectors
# 提示：gensim.models.keyedvectors.KeyedVectors.load_word2vec_format
####### your code ~1 line ######## 
wv_model = 
####### your code end ########

In [117]:
# 取出词语对应的词向量。
vec = wv_model[['糟糕','有趣','喜欢']]
print('三个词的词向量矩阵的维度是：', vec.shape,'。')
print('-------------------------------我是分隔符------------------------')
# 计算两个词的相似程度。
print('喜欢和喜爱的余弦相似度是：', wv_model.similarity('喜欢', '喜爱'),'。')
print('-------------------------------我是分隔符------------------------')
# 得到和某个词比较相关的词的列表
sim1 = wv_model.most_similar('有趣',topn=10)
for key in sim1:
    print('和有趣比较相关的词有',key[0],'，余弦距离是：',key[1])

三个词的词向量矩阵的维度是： (3, 100) 。
-------------------------------我是分隔符------------------------
喜欢和喜爱的余弦相似度是： 0.56644905 。
-------------------------------我是分隔符------------------------
和有趣比较相关的词有 强大 ，余弦距离是： 0.7989455461502075
和有趣比较相关的词有 复杂 ，余弦距离是： 0.7865893244743347
和有趣比较相关的词有 好玩 ，余弦距离是： 0.7752323746681213
和有趣比较相关的词有 有用 ，余弦距离是： 0.7646296620368958
和有趣比较相关的词有 有意思 ，余弦距离是： 0.7602453231811523
和有趣比较相关的词有 深刻 ，余弦距离是： 0.7590806484222412
和有趣比较相关的词有 简单 ，余弦距离是： 0.7489534020423889
和有趣比较相关的词有 熟悉 ，余弦距离是： 0.7469128370285034
和有趣比较相关的词有 单纯 ，余弦距离是： 0.74482262134552
和有趣比较相关的词有 迷人 ，余弦距离是： 0.7436302304267883


In [133]:
%%time 
#%%time 显示整个cell的运行时间
key_word = '完美'
sim1 = wv_model.most_similar(key_word,topn=10)
for key in sim1:
    print('和'+ key_word +'比较相关的词有',key[0],'，余弦距离是：',key[1])

和完美比较相关的词有 美妙 ，余弦距离是： 0.7493830323219299
和完美比较相关的词有 色彩 ，余弦距离是： 0.7492667436599731
和完美比较相关的词有 独特 ，余弦距离是： 0.7221761345863342
和完美比较相关的词有 丰富 ，余弦距离是： 0.7146338820457458
和完美比较相关的词有 拥有 ，余弦距离是： 0.712502121925354
和完美比较相关的词有 惬意 ，余弦距离是： 0.7077858448028564
和完美比较相关的词有 具有 ，余弦距离是： 0.7062147259712219
和完美比较相关的词有 曼妙 ，余弦距离是： 0.7031504511833191
和完美比较相关的词有 结合 ，余弦距离是： 0.702284038066864
和完美比较相关的词有 呈现 ，余弦距离是： 0.6994835138320923
CPU times: user 11.6 ms, sys: 1.56 ms, total: 13.2 ms
Wall time: 2.28 ms


### 5.Annoy加速查询

Annoy教程 [github链接](https://github.com/spotify/annoy)

官方demo:
```
from annoy import AnnoyIndex
import random
f = 40  # 向量的维度
t = AnnoyIndex(f, 'angular')
for i in range(1000):
    v = [random.gauss(0, 1) for z in range(f)] # 随机生成一个向量
    t.add_item(i, v) # 添加向量到t中
t.build(10) #  10颗树，树越多精度越高
t.save('test.ann') # 保存模型
u = AnnoyIndex(f, 'angular')
u.load('test.ann') # super fast, will just mmap the file
print(u.get_nns_by_item(0, 1000)) # 查询1000近邻的数据
```

In [119]:
# 安装annoy
!pip install annoy==1.17.0

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [120]:
from annoy import AnnoyIndex
import os
from tqdm import tqdm
vector_file_path = './wv_model/word2vec_zh.txt'
save_file = './wv_model/word2vec_zh.annoy'
def get_word_vec():
    with open(vector_file_path,'r',encoding='utf-8') as f:
        for i, line in enumerate(f):
            # 掠过第一行的表头
            if i == 0:
                continue
            # 去除两端换行符
            line=line.strip()  
            line=line.split(' ')
            word = line[0]
            vec = [float(i) for i in line[1:]]
            assert len(vec)==param['vector_size']
            yield word, vec

### 问题五：实现annoy建立索引的过程
可以根据官方的demo作答

In [121]:
def build_annoy_index(emb_size=param['vector_size'],metric='angular',num_trees=100):
    # 提示：新建AnnoyIndex对象t，并指定参数
    ####### your code ~1 line ######## 
    t=
    ####### your code end ######## 
    num=-1
    for word,vec in get_word_vec():
        num+=1
        # 提示：遍历往t中添加向量
        ####### your code ~1 line ######## 
        
        ####### your code end ######## 
    t.build(num_trees)
    t.save(save_file)

In [122]:
build_annoy_index()

In [123]:
annoy_model = AnnoyIndex(param['vector_size'],'angular')
annoy_model.load(save_file)

True

In [124]:
# 通过wv_model拿到词与id、id与词的对应关系
word2id = wv_model.key_to_index
id2word = wv_model.index_to_key

In [125]:
key_word = '完美'

可以看到使用`Annoy`比使用`most_similar`函数查询Top 10相似词的速度较快些

In [130]:
%%time
topn = 10
raw_res = annoy_model.get_nns_by_item(i=word2id[key_word],n=topn,include_distances=True)
for key, sim in zip(*raw_res):
    print('和'+ key_word +'比较相关的词有',id2word[key],'，距离是：',sim)

和完美比较相关的词有 完美 ，距离是： 0.00013219143147580326
和完美比较相关的词有 美妙 ，距离是： 0.7079789638519287
和完美比较相关的词有 色彩 ，距离是： 0.708142876625061
和完美比较相关的词有 独特 ，距离是： 0.7454178333282471
和完美比较相关的词有 丰富 ，距离是： 0.7554683089256287
和完美比较相关的词有 拥有 ，距离是： 0.7582848072052002
和完美比较相关的词有 惬意 ，距离是： 0.7644792199134827
和完美比较相关的词有 曼妙 ，距离是： 0.7705186605453491
和完美比较相关的词有 结合 ，距离是： 0.77164226770401
和完美比较相关的词有 呈现 ，距离是： 0.7752633094787598
CPU times: user 928 µs, sys: 776 µs, total: 1.7 ms
Wall time: 873 µs
